In [ ]:
import os
import glob
from PIL import Image
from tqdm import tqdm
import random

import pandas as pd
import numpy as np

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision as tv

import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

# Configuration 

In [ ]:
ROOT_DIR = ""
IMG_SIZE = 64
BATCH_SIZE = 128
LATENT_DIMS = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset preparation

In [ ]:
train_csv = pd.read_csv(ROOT_DIR + "Train.csv")
test_csv = pd.read_csv(ROOT_DIR + "Test.csv")

train_files = train_csv[["Path", "ClassId"]]
test_files = test_csv[["Path", "ClassId"]]

In [ ]:
tfms = tv.transforms.Compose([tv.transforms.Resize((IMG_SIZE, IMG_SIZE)), tv.transforms.ToTensor()])
filenames = [os.path.join(dirpath,filename) for dirpath, _, filenames in os.walk(ROOT_DIR + "Train/") for filename in filenames if filename.endswith('.png')]

In [ ]:
# Load data into memory

file_arr = []
for i in tqdm(range(len(filenames))):
    image = Image.open(filenames[i])
    tens = tfms(image)
    conv_filename = filenames[i].split("gtsrb/")[-1]
    class_id = int(train_files[train_files["Path"] == conv_filename]["ClassId"].astype(int))
    tens_id_arr = [tens, class_id]
    file_arr.append(tens_id_arr)

In [ ]:
# make sure that classes are mixed before splitting array into train and validation set

random.shuffle(file_arr)

train_files = file_arr[:-1000]
valid_files = file_arr[-1000:]

In [ ]:
class TSDataset(Dataset):
    def __init__(self, files, transform=None):
        self.files = files
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        x = self.files[idx][0]
        label = self.files[idx][1]
            
        return x, label

In [ ]:
training_data = TSDataset(train_files, tfms)
valid_data = TSDataset(valid_files, tfms) 

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
# shuffle = false to be able to compare output(-improvements) during training
valid_dataloader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Classifier architecture

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv3 = nn.Conv2d(20, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 43)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv3(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)


class UnFlatten(nn.Module):
    def forward(self, input, size=1024):
        return input.view(input.size(0), size, 1, 1)

In [ ]:
# https://www.kaggle.com/code/muhammad4hmed/anime-vae/notebook

class CVAE(nn.Module):
    def __init__(self, image_channels=3, h_dim=1024, z_dim=16):
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2),
            nn.ReLU(),
            Flatten()
        )
        
        self.signclass_embedding = nn.Embedding(43, 10)
        
        self.h2mu = nn.Linear(h_dim, z_dim)
        self.h2sigma = nn.Linear(h_dim, z_dim)
        self.z2h = nn.Linear(z_dim + 10, h_dim)
        
        self.decoder = nn.Sequential(
            UnFlatten(),
            nn.ConvTranspose2d(h_dim, 128, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=6, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, image_channels, kernel_size=6, stride=2),
            nn.Sigmoid(),
        )
        
    # Enforce latent space well-formedness by jinecting random gaussian noise    
    def reparameterize(self, mu, logvar):          
        std = logvar.mul(0.5).exp_()
        eps = torch.randn(*mu.size()).to(DEVICE)
        z = mu + std * eps
        return z
    
    def bottleneck(self, h, label):
        mu = self.h2mu(h)
        logvar = self.h2sigma(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar
        
    def encode(self, x, label):
        return self.bottleneck(self.encoder(x), label)[0]

    def decode(self, z):
        return self.decoder(self.z2h(z))
    
    def forward(self, x, label):
        h = self.encoder(x)
        z_small, mu, logvar = self.bottleneck(h, label)     
        signclass = self.signclass_embedding(label.long())
        signclass = signclass.squeeze(dim=1)
        z_small_cat = torch.cat([z_small, signclass], dim=1)
        z = self.z2h(z_small_cat)
        return self.decoder(z), mu, logvar, z_small, z

In [ ]:
# Ensemble architecture (combining cvae and classifier)

class Ensemble(nn.Module):
    def __init__(self, embeddings, upscaler, decoder, classifier):
        super(Ensemble, self).__init__()
        self.embeddings = embeddings
        self.upscaler = upscaler
        self.decoder = decoder
        self.classifier = classifier
        
    def forward(self, z, label):
        enc_label = self.embeddings(label.long())
        enc_label = enc_label.squeeze(dim=1)
        x = torch.cat((z, enc_label), dim=1)
        x = self.upscaler(x)
        x = self.decoder(x)
        x = self.classifier(x)
        return x
    
    def get_img(self, z, label):
        enc_label = self.embeddings(label.long())
        x = torch.cat((z, enc_label), dim=1)
        x = self.upscaler(x)
        x = self.decoder(x)
        return x

In [ ]:
# Load pre-trained models for classifier and cvae

classifier = Classifier()
cvae = CVAE()

classifier.eval()
cvae.eval()

classifier.load_state_dict(torch.load("..."))
cvae.load_state_dict(torch.load('...'))

classifier.to(DEVICE)
cvae.to(DEVICE);

In [ ]:
# Load cvae and classifier into ensemble

embeddings, upscaler, decoder = cvae.extract_model()
ensemble = Ensemble(embeddings, upscaler, decoder, classifier)
ensemble.to(DEVICE);

In [ ]:
# Calculate adversarial example
# https://adversarial-ml-tutorial.org/adversarial_training/

def pgd_linf(model, X, y, epsilon, alpha, num_iter):
    delta = torch.zeros_like(X, requires_grad=True)
    for t in range(num_iter):
        pred = model(X + delta, y)
        loss = nn.CrossEntropyLoss()(pred, y.squeeze(dim=1))
        loss.backward(retain_graph=True)
        delta.data = (delta + alpha*delta.grad.detach().sign()).clamp(-epsilon,epsilon)
        delta.grad.zero_()
    return delta.detach()

In [ ]:
cvae_data, cvae_labels = next(iter(train_dataloader))
cvae_data, cvae_labels = cvae_data.to(DEVICE), cvae_labels.to(DEVICE)
cvae_labels = cvae_labels.unsqueeze(dim=1)
cvae_data.shape, cvae_labels.shape

In [ ]:
recon_batch, mu, logvar, data, z = cvae(cvae_data, cvae_labels)

# Visualize the number of adversaries per class

In [ ]:
# Sample 1024 uniformly distributed scene vectors

uniform_dist = (-2-2) * torch.rand_like(mu) + 2

In [ ]:
# For all sign classes, calculate 1024 adversarial perturbations from random starting points in [-2,2]

all_danger = torch.zeros((43, 1024))

for i in tqdm(range(43)):
    cvae_labels = [i]
    cvae_labels = torch.Tensor(cvae_labels).unsqueeze(dim=1)
    cvae_labels = cvae_labels.expand(BATCH_SIZE, -1)
    cvae_labels = cvae_labels.type(torch.LongTensor)
    cvae_labels = cvae_labels.to(DEVICE)

    mu2 = uniform_dist
    recon_batch, mu, logvar, data, z = cvae(cvae_data, cvae_labels)
    delta = pgd_linf(ensemble, mu2, cvae_labels, epsilon=0.2, alpha=2e-2, num_iter=50)
    yp = ensemble(mu2 + delta, cvae_labels)
    prob_orig = F.softmax(yp)

    danger_array = prob_orig.gather(1, cvae_labels).squeeze(dim=1)
    danger_array = danger_array.to("cpu").detach()
    danger_array = danger_array.unsqueeze(dim=0)
    
    all_danger[i] = danger_array

In [ ]:
# Order the previsouly calculated p(correct_class) 

ordered = torch.zeros((43, 1024))
danger_order = torch.argsort(all_danger, dim=1)
top = danger_order[:,:1024]

for i in range(43):
    top_danger = all_danger[i, top[i]]
    ordered[i] = top_danger

In [ ]:
# Visualize the effect that adversarial perturbations have on the classifier's performance

selected_class = 10
plt.plot(ordered[selected_class], label="")
plt.show()

# Failure Mode Computing

In [ ]:
# Option-1: Create label-vector (LOCAL)

selected_class = 10

cvae_labels = [selected_class]
cvae_labels = torch.Tensor(cvae_labels).unsqueeze(dim=1)
cvae_labels = cvae_labels.expand(BATCH_SIZE, -1)
cvae_labels = cvae_labels.type(torch.LongTensor)
cvae_labels = cvae_labels.to(DEVICE)

In [ ]:
# Option-2: Create label-vector (GLOBAL)

cvae_labels = []
cvae_labels.extend(range(0, 43))
cvae_labels = torch.Tensor(cvae_labels).unsqueeze(dim=1)
cvae_labels = cvae_labels.type(torch.LongTensor)
cvae_labels = cvae_labels.to(DEVICE)

In [ ]:
# Option-1: Calculate p(correct_class) for BATCH_SIZE adversaries (LOCAL)

mu2 = uniform_dist
delta = pgd_linf(ensemble, mu2, cvae_labels, epsilon=0.2, alpha=2e-2, num_iter=50)
yp = ensemble(mu2 + delta, cvae_labels)
prob_orig = F.softmax(yp)

mu_adv = mu2 + delta

danger_array = prob_orig.gather(1, cvae_labels).squeeze(dim=1)
l2_array = torch.linalg.norm(mu2, dim=1, ord=2)

In [ ]:
# Option-2: Calculate p(correct_class) for BATCH_SIZE adversaries (GLOBAL)

danger_array = []
l2_array = []
delta_array = []

for i in tqdm(range(BATCH_SIZE))
    mu2 = uniform_dist[i].expand(43, -1)
    delta = pgd_linf(ensemble, mu2, cvae_labels, epsilon=0.2, alpha=2e-2, num_iter=50)
    yp = ensemble(mu2 + delta, cvae_labels)
    prob_orig = F.softmax(yp)
    
    danger = sum(prob_orig.gather(1, cvae_labels))[0].item()/43
    l2_norm = torch.linalg.norm(mu[i], dim=0, ord=2)
    
    l2_array.append(l2_norm.item())
    danger_array.append(danger)
    delta_array.append(delta)

In [ ]:
# Select most dangerous 200 styles (LOCAL & GLOBAL) 

d_tensor = torch.Tensor(danger_array)
l2_tensor = torch.Tensor(l2_array)
normed_tensor = d_tensor*l2_tensor

danger_order = torch.argsort(d_tensor, dim=0)
top = danger_order[:200]

mu_sel = mu[top]
mu_sel = mu_sel.to("cpu").detach().numpy() 
d_sel = d_tensor[top]

In [ ]:
# Apply k-Means to the 200 most dangerous perturbed scenes

k = 4
kmeans = KMeans(n_clusters=k, random_state=42).fit(mu_sel)
labels = kmeans.labels_
clusts = torch.Tensor(kmeans.cluster_centers_)

In [ ]:
def plot_images(X, y, yp, N):
    fig = plt.figure()
    for j in range(N):
        a = fig.add_subplot(1,4,j+1)
        a.imshow(X[j])
        a.set_axis_off()
    plt.tight_layout()

In [ ]:
# Plot the k failure modes

yp = ensemble(clusts, cvae_labels[:k])

imgs = ensemble.get_img(clusts, cvae_labels[:k].squeeze(dim=1))
imgs = imgs.detach().cpu().numpy()
imgs = imgs.transpose(0, 2, 3, 1)
plot_images(imgs, cvae_labels[:k], yp, 4)

# t-SNE Visualization

In [ ]:
tsne_results = TSNE(n_components=2, verbose=1, metric='euclidean').fit_transform(mu_sel);

In [ ]:
# Assign color to p(correct_label); 0.00 = red, 1.00 = green, ...

colorscale = ["#F50E00", "#E62D00", "#D94800", "#C66D00", "#B98700", "#B98700", "#A6AE00", "#97CD00", "#88EB00", "#80FA00"]
def assign_color(prob):
    pct_val = int(((prob*100)/10)-1)
    pct_val = max(pct_val, 0)
    return colorscale[pct_val]

In [ ]:
# Plot images in t-SNE grid

def plot_images_in_2d(x, y, image_idxs, axis=None, zoom=1):
    if axis is None:
        axis = plt.gca()
    x, y = np.atleast_1d(x, y)
    for x0, y0, idx in zip(x, y, image_idxs):
        style = uniform_dist[idx]
        style = style.unsqueeze(dim=0)
        # Select a class
        label = torch.Tensor([10])
        
        img = ensemble.get_img(style, label.squeeze(dim=1))
        imgs = img.detach().cpu().numpy()
        imgs = imgs.transpose(0, 2, 3, 1)
        imgs = imgs[0]
        imgs = OffsetImage(imgs, zoom=zoom)
        anno_box = AnnotationBbox(imgs, (x0, y0),
                                  xycoords='data',
                                  frameon=False)
        axis.annotate("{:.2f}".format(d_tensor[idx]), color=assign_color(d_tensor[idx]), xy=(x0-17,(y0+40)))
        axis.add_artist(anno_box)
    axis.update_datalim(np.column_stack([x, y]))
    axis.autoscale()

In [ ]:
# Create t-SNE-grid
# https://github.com/PracticalDL/Practical-Deep-Learning-Book/blob/master/code/chapter-4/2-similarity-search-level-1.ipynb

def tsne_to_grid_plotter_manual(x, y, style_idxs):

    S = 2000
    s = 100
    x = (x - min(x)) / (max(x) - min(x))
    y = (y - min(y)) / (max(y) - min(y))
    
    x_values = []
    y_values = []
    idx_plot = []
    x_y_dict = {}

    for i, idx in enumerate(style_idxs):
        a = np.ceil(x[i] * (S - s))
        b = np.ceil(y[i] * (S - s))
        a = int(a - np.mod(a, s))
        b = int(b - np.mod(b, s))
        
        if str(a) + "|" + str(b) in x_y_dict:
            continue
            
        x_y_dict[str(a) + "|" + str(b)] = 1
        x_values.append(a)
        y_values.append(b)
        idx_plot.append(idx)
        
    fig, axis = plt.subplots()
    fig.set_size_inches(22, 22, forward=True)
    plot_images_in_2d(x_values, y_values, idx_plot, zoom=.58, axis=axis)
    plt.xticks([])
    plt.yticks([])
    plt.savefig('...')
    plt.show()
    
    return (x_values, y_values, idx_plot)

In [ ]:
# Return and plot t-SNE-results

x, y, idxs = tsne_to_grid_plotter_manual(tsne_results[:, 0], tsne_results[:, 1], top)